In [3]:
import pandas as pd

Week of September 29th, exploring python filters
imported course catalog dataset

Week of October 6th, exploring python filters
delete unnecesary columns from main dataframe
concatenate GPA dataset

In [4]:
#import datasets

#import course catalog dataset
df = pd.read_csv('2025-sp.csv') #df for data frame

#import course GPA dataset
df_GPA = pd.read_csv('uiuc-gpa-dataset.csv')

#import teacher rating dataset
df_Rate = pd.read_csv('uiuc-tre-dataset.csv')

In [94]:
#filter practice: filters that could be applied to dataset
filtSubject = df['Subject'].str.contains('CS')
filtSubject = df['Subject'] == 'CS'
filtNum = df['Number'] == 173
filtBoth = filtSubject & filtNum

#applying filters
filt = filtBoth
#df[filt].head(20)
df.loc[filt, ['Subject', 'Number', 'Start Time', 'Days of Week',"Building"]].head(10)

,Subject,Number,Start Time,Days of Week,Building
3094,CS,173,09:30 AM,TR,Campus Instructional Facility
3095,CS,173,09:30 AM,TR,Campus Instructional Facility
3096,CS,173,03:30 PM,TR,Campus Instructional Facility
3097,CS,173,03:30 PM,TR,Campus Instructional Facility
3098,CS,173,02:00 PM,TR,Campus Instructional Facility
3099,CS,173,02:00 PM,TR,Campus Instructional Facility


In [5]:
#modify name function
def modify_Name(name):
    if not isinstance(name, str):
        return name  # Handle non-string values
    
    parts = name.split(',')
    last_name = parts[0].strip() # Get the last name
    first_initial = parts[1].strip()[:1].upper()  # Get the first initial
    return last_name + ', ' + first_initial

In [ ]:
#DF cleaning and modification

#delete unnecesary columns from main dataframe
drop_cols = ['Term', 'Section Info','Schedule Information', 'Section Title', 'Enrollment Status','Status Code', 'Section Status', 'Section Credit Hours']
df.drop(drop_cols, axis=1, inplace=True) 
df['Primary Instructor'] = df['Instructors'].apply(lambda x: str(x).split(';')[0]) #add column for primary instructor
df.rename(columns={'Type Code': 'Sched Type'}, inplace=True) #rename column to match GPA dataset

# Chaging the last name to only first initial (to match main DF)
df_GPA['Primary Instructor'] = df_GPA['Primary Instructor'].apply(lambda name: modify_Name(name))

# create mean_df_profBased_GPA + mean_df_classBased_GPA; mean GPA by professor for each class / only by class
group_cols_wProf = ['Subject', 'Number', 'Sched Type', 'Primary Instructor']
group_cols = ['Subject', 'Number', 'Sched Type']
merge_cols = ['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+','C', 'C-', 'D+', 'D', 'D-', 'F', 'W', 'Students']
# by prof + class
mean_df_profBased_GPA = df_GPA.groupby(group_cols_wProf, as_index = False)[merge_cols].mean().round(2)
mean_df_profBased_GPA['Mean Grade By Professor (A+..F,W,Students)'] = mean_df_profBased_GPA[merge_cols].values.tolist()
mean_df_profBased_GPA.drop(merge_cols, axis=1, inplace=True) 
# by only class
mean_df_classBased_GPA = df_GPA.groupby(group_cols, as_index = False)[merge_cols].mean().round(2)
mean_df_classBased_GPA['Mean Grade By Class (A+..F,W,Students)'] = mean_df_classBased_GPA[merge_cols].values.tolist()
mean_df_classBased_GPA.drop(merge_cols, axis=1, inplace=True) 

mean_df_classBased_GPA.head(10)

,Subject,Number,Sched Type,"Mean Grade By Class (A+..F,W,Students)"
0,AAS,100,DIS,"[4.27, 11.72, 4.13, 2.51, 2.62, 1.2, 0.54, 0.5..."
1,AAS,100,LCD,"[5.57, 10.07, 4.79, 3.07, 2.71, 1.36, 0.21, 0...."
2,AAS,100,OD,"[4.62, 11.5, 3.17, 1.75, 2.17, 0.62, 0.71, 0.8..."
3,AAS,105,LCD,"[10.0, 7.0, 3.0, 2.0, 0.0, 0.0, 0.0, 1.0, 0.0,..."
4,AAS,105,ONL,"[13.0, 13.0, 0.0, 1.0, 4.0, 1.0, 0.0, 0.0, 0.0..."
5,AAS,120,DIS,"[1.0, 6.31, 5.69, 5.46, 5.0, 2.46, 1.69, 1.62,..."
6,AAS,120,LCD,"[2.4, 9.8, 4.8, 1.8, 3.0, 2.6, 1.2, 1.0, 1.0, ..."
7,AAS,120,ONL,"[4.0, 2.0, 15.0, 8.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
8,AAS,200,LCD,"[3.43, 17.0, 4.29, 2.43, 2.14, 1.29, 0.14, 0.2..."
9,AAS,200,ONL,"[4.0, 4.0, 13.0, 7.0, 0.0, 1.0, 0.0, 0.0, 0.0,..."


Data Issue:
Right now, the issue with the merged dataset is that the df_GPA contains GPA information for the past 15 years, starting from 2010 to 2015.
A couple possibilites here:
    1. only take GPA data from the current year
        - missing out on a lot of data here
    2. take the most recent GPA
        - again, missing out on data here 
        - (info could be provided in interface (i.e. data from 2022))
    3. take the average of the GPA's for each professor
        - changing class structure could be inacurate representation
    4. approaching peoples preferences differently:
        - the algorithm could go by average class GPA instead of Professor-specific GPA 
            - if said student doesn't care about professor nuiances 
                - this would be for further on when schedule generator could pick classes as well outside of generating a schedule based purely on inputs

What is shown in the code below (there is something wrong with the merge function):
Somehow, the df.merge is not actually doing a left merge, but actually a outer merge.
This is how we are getting duplicate CRNs and duplicate GPA numbers

In [7]:
# merge df with both df_GPA
df = df.merge(mean_df_profBased_GPA, how='left', on=group_cols_wProf)
df = df.merge(mean_df_classBased_GPA, how='left', on=group_cols)
df.head()

,Year,YearTerm,Subject,Number,Name,Description,Credit Hours,Degree Attributes,CRN,Section,...,Sched Type,Start Time,End Time,Days of Week,Room,Building,Instructors,Primary Instructor,"Mean Grade By Professor (A+..F,W,Students)","Mean Grade By Class (A+..F,W,Students)"
0,2025,2025-sp,AAS,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,"Social & Beh Sci - Soc Sci, and Cultural Studi...",69781,AB,...,LCD,02:00 PM,03:20 PM,MW,304,Noyes Laboratory,"Geng, C","Geng, C",NaN,"[5.57, 10.07, 4.79, 3.07, 2.71, 1.36, 0.21, 0...."
1,2025,2025-sp,AAS,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,"Social & Beh Sci - Soc Sci, and Cultural Studi...",30107,AD1,...,DIS,09:00 AM,09:50 AM,F,1030,"Literatures, Cultures, & Ling","Siglos, D;Wang, Y","Siglos, D",NaN,"[4.27, 11.72, 4.13, 2.51, 2.62, 1.2, 0.54, 0.5..."
2,2025,2025-sp,AAS,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,"Social & Beh Sci - Soc Sci, and Cultural Studi...",41729,AD2,...,DIS,10:00 AM,10:50 AM,F,1030,"Literatures, Cultures, & Ling","Siglos, D;Wang, Y","Siglos, D",NaN,"[4.27, 11.72, 4.13, 2.51, 2.62, 1.2, 0.54, 0.5..."
3,2025,2025-sp,AAS,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,"Social & Beh Sci - Soc Sci, and Cultural Studi...",43832,AD3,...,DIS,11:00 AM,11:50 AM,F,1030,"Literatures, Cultures, & Ling","Kim, G;Siglos, D","Kim, G","[11.5, 5.5, 1.5, 0.0, 4.0, 0.5, 0.0, 0.0, 0.5,...","[4.27, 11.72, 4.13, 2.51, 2.62, 1.2, 0.54, 0.5..."
4,2025,2025-sp,AAS,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,"Social & Beh Sci - Soc Sci, and Cultural Studi...",48232,AD4,...,DIS,12:00 PM,12:50 PM,F,1030,"Literatures, Cultures, & Ling","Kim, G;Siglos, D","Kim, G","[11.5, 5.5, 1.5, 0.0, 4.0, 0.5, 0.0, 0.0, 0.5,...","[4.27, 11.72, 4.13, 2.51, 2.62, 1.2, 0.54, 0.5..."


In [9]:
df.columns

Index(['Year', 'YearTerm', 'Subject', 'Number', 'Name', 'Description',
       'Credit Hours', 'Degree Attributes', 'CRN', 'Section', 'Part of Term',
       'Type', 'Sched Type', 'Start Time', 'End Time', 'Days of Week', 'Room',
       'Building', 'Instructors', 'Primary Instructor',
       'Mean Grade By Professor (A+..F,W,Students)',
       'Mean Grade By Class (A+..F,W,Students)'],
      dtype='object')

In [10]:
df_Rate['Year'] = df_Rate['term'].str.extract(r'(\d{4})').astype(float) #extracts the year from term
df_Rate = df_Rate[df_Rate['Year'] >= 2010]
df_Rate.drop(columns='Year', inplace=True)
df_Rate.head(10)

,term,unit,lname,fname,role,ranking,course
26098,fa2010,ACCOUNTANCY,BECK,J,Instructor,Excellent,415
26099,fa2010,ACCOUNTANCY,BOBBITT,L,TA,Outstanding,202
26100,fa2010,ACCOUNTANCY,BRADLEY,K,TA,Excellent,202
26101,fa2010,ACCOUNTANCY,CHANDLER,J,Instructor,Excellent,500
26102,fa2010,ACCOUNTANCY,CHEN,X,Instructor,Excellent,302
26103,fa2010,ACCOUNTANCY,ELLIOTT,W,Instructor,Excellent,303
26104,fa2010,ACCOUNTANCY,FELLER,A,Instructor,Excellent,304
26105,fa2010,ACCOUNTANCY,GONG,J,Instructor,Excellent,301
26106,fa2010,ACCOUNTANCY,HAMILTON,L,TA,Excellent,201
26107,fa2010,ACCOUNTANCY,HOLDER,D,Instructor,Excellent,312


^^ rating also provides a tricky challenge - how do you define if the ranknig for a professor is good? does it count indefinitely after you get a ranking? or is it better to get more rankings? wouldnt that put teachers who are new at a disadvatange? is outstanding double as good as excellent? How do you quantify this?

In [29]:
import ratemyprofessor as rmp
UIUC = rmp.School(1112) #1112 is UIUC's school ID in RMP
print(UIUC.id) #prints the name of the school
def getGMP(profName):
    prof = rmp.get_professor_by_school_and_name(UIUC, profName)
    if prof is not None:
        return prof.rating
    else:
        return -1
getGMP("Nathan D")

ValueError: Invalid school id or bad request.

In [ ]:
import requests
import json
import math

def getProfRMP(profName):
    page = requests.get("http://www.ratemyprofessors.com/")#search/professors/1112?q=" + profName.replace(' ', '%20'))
    temp_jsonpage = json.loads(page.content)


class RateMyProfScraper:
        def __init__(self,schoolid):
            self.UniversityId = schoolid
            self.professorlist = self.createprofessorlist()
            self.indexnumber = False

        def createprofessorlist(self):#creates List object that include basic information on all Professors from the IDed University
            tempprofessorlist = []
            num_of_prof = self.GetNumOfProfessors(self.UniversityId)
            num_of_pages = math.ceil(num_of_prof / 20)
            i = 1
            while (i <= num_of_pages):# the loop insert all professor into list
                page = requests.get("http://www.ratemyprofessors.com/filter/professor/?&page=" + str(
                    i) + "&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid=" + str(
                    self.UniversityId))
                temp_jsonpage = json.loads(page.content)
                temp_list = temp_jsonpage['professors']
                tempprofessorlist.extend(temp_list)
                i += 1
            return tempprofessorlist

        def GetNumOfProfessors(self,id):  # function returns the number of professors in the university of the given ID.
            page = requests.get(
                "http://www.ratemyprofessors.com/filter/professor/?&page=1&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid=" + str(
                    id))  # get request for page
            temp_jsonpage = json.loads(page.content)
            num_of_prof = temp_jsonpage[
                              'remaining'] + 20  # get the number of professors at William Paterson University
            return num_of_prof

        def SearchProfessor(self, ProfessorName):
            self.indexnumber = self.GetProfessorIndex(ProfessorName)
            self.PrintProfessorInfo()
            return self.indexnumber

        def GetProfessorIndex(self,ProfessorName):  # function searches for professor in list
            for i in range(0, len(self.professorlist)):
                if (ProfessorName == (self.professorlist[i]['tFname'] + " " + self.professorlist[i]['tLname'])):
                    return i
            return False  # Return False is not found

        def PrintProfessorInfo(self):  # print search professor's name and RMP score
            if self.indexnumber == False:
                print("error")
            else:
                print(self.professorlist[self.indexnumber])

        def PrintProfessorDetail(self,key):  # print search professor's name and RMP score
            if self.indexnumber == False:
                print("error")
                return "error"
            else:
                print(self.professorlist[self.indexnumber][key])
                return self.professorlist[self.indexnumber][key]

In [12]:
UIUC = RateMyProfScraper(1112)
def get_RMP_index(ProfessorName):
    RMPprofindex = UIUC.GetProfessorIndex(ProfessorName)
    if RMPprofindex is not False:
        return UIUC.professorlist[RMPprofindex]["overall rating"]
    else:
        return -1

JSONDecodeError: Expecting value: line 1 column 1 (char 0)